# Introducing Row Swapping and the P Matrix in LU Decomposition

So far, we have seen **LU decomposition without row swaps**, where a matrix $A$ is factored as:

$$
A = LU
$$

where
- $L$ is lower triangular (with ones on the diagonal),
- $U$ is upper triangular.

However, not all matrices can be decomposed easily without **row swapping**. Sometimes, the pivot (the number we divide by during elimination) is zero or nearly zero, and **swapping rows** is necessary for correctness and numerical stability.

To handle this, we introduce a **permutation matrix** $P$, so that:

$$
PA = LU
$$

where $P$ reorders the rows of $A$ to ensure good pivots.

---

## What Is a Permutation Matrix?

- A **permutation matrix** $P$ is just a matrix that **swaps rows**.
- $P$ is square, with exactly one `1` in each row and column, and `0`s elsewhere.
- Important property: **$P^{-1} = P^T$**.  ($P$ is an orthonormal matrix)


Applying $P$ to $A$ (i.e., multiplying $P \times A$) reorders the rows of $A$.

---

## Example: 2×2 Matrix With a Needed Row Swap

Suppose:

$$
A = \begin{pmatrix} 0 & 2 \\ 1 & 2 \end{pmatrix}
$$

Notice:
- The pivot we want at the top left (position (0,0)) is `0`, which would cause division problems.
- So we **swap row 0 and row 1**.

The **permutation matrix** is:

$$
P = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}
$$

Applying $P$ to $A$ gives:

$$
PA = \begin{pmatrix} 1 & 2 \\ 0 & 2 \end{pmatrix}
$$

Now we can proceed normally with LU decomposition:

- $L = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$  
- $U = \begin{pmatrix} 1 & 2 \\ 0 & 2 \end{pmatrix}$

Thus:

$$
PA = LU
$$

---

## Why Swapping Is Important: Near Zero Pivot Example

Suppose:

$$
A = \begin{pmatrix} 1 \times 10^{-12} & 2 \\ 1 & 2 \end{pmatrix}
$$

Without swapping, the first pivot is $1 \times 10^{-12}$, an extremely tiny number.  
- Dividing by such a small number can lead to **huge rounding errors**.
- **Better strategy:** Swap the rows first!

Using the same permutation matrix:

$$
P = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}
$$

After swapping:

$$
PA = \begin{pmatrix} 1 & 2 \\ 1 \times 10^{-12} & 2 \end{pmatrix}
$$

Now, the pivot is a nice large number (`1`) — **much better for stability**.

---

## What `scipy.linalg.lu` Returns

The function:

```python
P, L, U = scipy.linalg.lu(A)
```

returns three matrices:
- $P$: permutation matrix (row swaps),
- $L$: lower triangular matrix (with ones on the diagonal),
- $U$: upper triangular matrix,

such that:

$$
A = PLU
$$

**Notes:**
- $P$ is a **full matrix**, not just a list of swap operations.
- It satisfies $P^{-1} = P^T$.
- This is different than the traditional $P$ discussed prior to this cell.  This is $P$ is the transpose of the $P$ that is typically used.

---

## Solving Linear Systems Using PLU

Suppose you want to solve:

$$
Ax = b
$$

Given the decomposition $PA = LU$, the procedure is:

1. **Apply $P$ to $b$:**
   $$
   Pb
   $$
2. **Solve $Ly = Pb$** using **forward substitution**.
3. **Solve $Ux = y$** using **backward substitution**.

This turns solving a complicated system into solving two **easy triangular systems**!

### If you're using `scipy.linalg.lu`

First get `P,L,U = lu(A)`.  Then use `P = P.T` to transpose `P` before following the steps above.

---

## Example Code: Solving a System Using PLU Decomposition



In [1]:
import numpy as np
from scipy.linalg import lu, solve_triangular

# Define matrix A and vector b
A = np.array([[0, 2],
              [1, 2]])
b = np.array([2, 4])


# Perform PLU decomposition
P, L, U = lu(A)

# IMPORTANT: the lu function returns P, L, U such that A = PLU, not PA = LU
# this means that the P returned by lu is the transpose of the P discussed above
# we need to transpose P to get the correct permutation matrix
P = P.T
# Note: In this case, P is a permutation matrix, so P.T is its inverse

# Step 1: Apply P to b
Pb = P @ b

# Step 2: Solve Ly = Pb (forward substitution)
y = solve_triangular(L, Pb, lower=True)

# Step 3: Solve Ux = y (backward substitution)
x = solve_triangular(U, y)

print("Solution x =", x)

Solution x = [2. 1.]


**Notes:**
- `solve_triangular(..., lower=True)` solves a lower triangular system.
- `solve_triangular(..., lower=False)` (default) solves an upper triangular system.
- This matches exactly the steps explained above: first $Ly = Pb$, then $Ux = y$.

---